### Voice Command Based Kiva Controller

#### Install Dependencies

In [ ]:
pip install speechrecognition transformers

#### Developing NLP Model

In [ ]:
!pip install accelerate -U --no-cache-dir
!pip install transformers[torch]
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch
from torch.utils.data import Dataset

# Example dataset
class CommandDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Example training data
texts = ["move forward", "turn left", "pick item", "check inventory", "start task"]
labels = [0, 0, 1, 1, 2]  # 0: navigation, 1: inventory_management, 2: task_delegation

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_dataset = CommandDataset(texts, labels, tokenizer)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

# Save the model
model.save_pretrained("command_model")
tokenizer.save_pretrained("command_model")

#### Loading the Model and Defining Utility Function

In [5]:
import speech_recognition as sr

model = DistilBertForSequenceClassification.from_pretrained("command_model")
tokenizer = DistilBertTokenizerFast.from_pretrained("command_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Define command categories
commands = {
    "navigation": ["move forward", "turn left", "turn right", "stop"],
    "inventory_management": ["pick item", "drop item", "check inventory"],
    "task_delegation": ["start task", "complete task", "status report"]
}

# Function to recognize speech and convert to text
def recognize_speech():
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"Recognized: {text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            return None
        except sr.RequestError:
            print("Request Error.")
            return None

def classify_command(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Map predicted class ID to command category
    command_categories = {0: "navigation", 1: "inventory_management", 2: "task_delegation"}
    label = command_categories.get(predicted_class_id, "unknown")
    print(f"Command Classification: {label}")
    return label


### Main Loop and Robot API

In [12]:
import requests

API_ENDPOINT = "http://127.0.0.1/robot-api"

def send_command_to_robot(command):
    response = requests.post(API_ENDPOINT, json={"command": command})
    if response.status_code == 200:
        print("Command executed successfully.")
    else:
        print("Failed to execute command.")

while True:
    text = recognize_speech()
    if text!="exit":
        label = classify_command(text)
        if label == "navigation":
            print("Processing navigation command...")
            send_command_to_robot(text)
        elif label == "inventory_management":
            print("Processing inventory management command...")
            send_command_to_robot(text)
        elif label == "task_delegation":
            print("Processing task delegation command...")
            send_command_to_robot(text)
        else:
            print("Unknown command. Please try again.")
    else:
      print("Exiting...")
      break

move left
Command Classification: navigation
Processing navigation command...
Command executed successfully.
pick bottle
Command Classification: inventory_management
Processing inventory management command...
Command executed successfully.
exit
Exiting...
